In [1]:
import mainCode as Haha
import tensorflow as tf

# basic data (ODs prediction)

In [2]:
getdata = Haha.BasicData(which_data='mine')
getdata.readOD(50, od_selected=None, filelist=['webScrapping/tgsc_odorant_1020.txt'], keep_odStr=True)
smiles_list = getdata.readSmiles(filelist=['webScrapping/tgsc_odorant_1020.txt'])
print(len(smiles_list))

getdata.which_data = 'odorless'
smiles_list_odorless = getdata.readSmiles(filelist=['webScrapping/tgsc_odorless_1020.txt']) 
print(len(smiles_list_odorless))

smiles_list.extend(smiles_list_odorless)
print(len(smiles_list))
temp = (len(smiles_list)-len(getdata.all_odStr)) * ['odorless']
getdata.all_odStr.extend(temp)
print(len(getdata.all_odStr))

number of all type odor descriptors = 1615
appear times >= 50 odor descriptors have 98
they are
['fruity', 'sweet', 'green', 'floral', 'woody', 'herbaceous', 'fresh', 'fatty', 'spicy', 'waxy', 'citrus', 'rose', 'nutty', 'oily', 'earthy', 'balsam', 'berry', 'tropical', 'sulfurous', 'aromatic', 'vegetable', 'apple', 'minty', 'powdery', 'roasted', 'meaty', 'pineapple', 'creamy', 'aldehydic', 'ethereal', 'musty', 'phenolic', 'leafy', 'honey', 'caramel', 'camphor', 'pungent', 'metallic', 'orange', 'winey', 'pear', 'melon', 'natural', 'banana', 'amber', 'buttery', 'jasmin', 'onion', 'cheese', 'musk', 'animal', 'coffee', 'lemon', 'coconut', 'pine', 'burnt', 'vanilla', 'peach', 'mushroom', 'tobacco', 'apricot', 'cooling', 'cocoa', 'garlic', 'violet', 'cherry', 'cinnamon', 'almond', 'grape', 'anise', 'medicinal', 'rum', 'chocolate', 'milky', 'dairy', 'alliaceous', 'grassy', 'lily', 'cooked', 'hyacinth', 'geranium', 'grapefruit', 'plum', 'terpene', 'skin', 'cedar', 'pepper', 'leather', 'lavender

In [3]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# ODs prediction

In [4]:
test = Haha.Transformer2OD_tada()
test.molData_atomicF(smiles_list, dist_value='exp', atomH=False, failed_mol=None, rseed=1)
test.odData(getdata.od_mat_ori, getdata.od_name)

Transformer2OD_tada
transformer2od
97 mol cannot compute distance, they are: [0, 13, 39, 87, 140, 643, 644, 741, 757, 766, 774, 785, 791, 804, 839, 842, 902, 908, 1184, 1328, 1342, 1345, 1346, 1379, 1529, 1530, 1781, 1868, 1887, 2067, 2078, 2080, 2216, 2222, 2233, 2240, 2246, 2256, 2262, 2264, 2270, 2439, 2444, 2493, 2505, 2601, 2622, 2664, 2692, 2713, 2763, 2767, 2775, 2822, 2844, 2859, 2868, 2873, 2897, 2961, 2988, 3107, 3437, 3511, 3522, 3565, 3736, 3746, 3765, 3805, 3865, 3881, 3921, 3995, 4082, 4083, 4153, 4199, 4202, 4204, 4246, 4254, 4267, 4268, 4272, 4297, 4299, 4303, 4304, 4357, 4383, 4418, 4419, 4421, 4431, 4436, 4451]
num_sample : 4365
mask_mat: (4365, 1, 1, 60)
mol_atom_feat.shape: (4365, 60, 71)
distance value type is: exp
all_adj.shape = : (4365, 60, 60)
all_dist.shape = : (4365, 60, 60)
od_shuffled.shape =  (4365, 98)
positive sample distribution in test set:
{'fruity': 242.0, 'sweet': 208.0, 'green': 189.0, 'floral': 147.0, 'woody': 107.0, 'herbaceous': 101.0, 'fresh': 

In [5]:
test.use_adj_dist = 'both'
hpara = {
            'num_heads': 8,
            'single_attn_dim': 30,
            'num_encoderLayer': 7,
            'num_decoderLayer': 2, 
            'learning_rate': 0.00002, 
            'temperature': 0.7
        }

record_dir='/tf/haha/code/logs/1114/od98-8_30_72'

weight_path = '/tf/haha/save_weight/1114/od98-8_30_72/'

In [8]:
test.sample_weight = True
test.modelTrain2(hpara, epochs=150, batch_size=32, record_dir=record_dir, save_threshold=0.3, save_path=weight_path, normal_init=False)

/tf/haha/code/logs/1114/od98-8_30_72
modelBuild2-temperature 0.700000
(3635, 1, 1)
(3635, 60, 71)
(3635, 60, 60)
(3635, 60, 60)
(3635, 1, 1, 60)
(3635, 98)
samples_weight using: fruity: pos2.344 neg1.000   samples_weight using: sweet: pos2.783 neg1.000   samples_weight using: green: pos2.981 neg1.000   samples_weight using: floral: pos3.789 neg1.000   samples_weight using: woody: pos5.388 neg1.000   samples_weight using: herbaceous: pos6.701 neg1.000   samples_weight using: fresh: pos6.851 neg1.000   samples_weight using: fatty: pos9.415 neg1.000   samples_weight using: spicy: pos9.916 neg1.000   samples_weight using: waxy: pos9.754 neg1.000   samples_weight using: citrus: pos12.218 neg1.000   samples_weight using: rose: pos11.890 neg1.000   samples_weight using: nutty: pos14.668 neg1.000   samples_weight using: oily: pos14.601 neg1.000   samples_weight using: earthy: pos14.943 neg1.000   samples_weight using: balsam: pos15.374 neg1.000   samples_weight using: berry: pos18.649 neg1.000

(<keras.engine.functional.Functional at 0x7fdc49c3df40>,
 {'avg': 0.33147895063370353,
  'avg_pcs': 0.30393450669184024,
  'avg_rc': 0.38927873991885964,
  'fruity': 0.6525096304239195,
  'sweet': 0.5200845580370965,
  'green': 0.5278450335554951,
  'floral': 0.5400593576167682,
  'woody': 0.5315315162179609,
  'herbaceous': 0.4147465461205074,
  'fresh': 0.3465346586890062,
  'fatty': 0.4733727955253869,
  'spicy': 0.3821656122217894,
  'waxy': 0.46153846882544874,
  'citrus': 0.45901639213711937,
  'rose': 0.5196850390522662,
  'nutty': 0.46428572051987355,
  'oily': 0.24000000161528587,
  'earthy': 0.26086956737228484,
  'balsam': 0.4646464674135407,
  'berry': 0.25531914559706875,
  'tropical': 0.37362637243154534,
  'sulfurous': 0.7173912948691098,
  'aromatic': 0.12345679368011246,
  'vegetable': 0.3218390746087659,
  'apple': 0.382022482159011,
  'minty': 0.43835615521189536,
  'powdery': 0.21739131015054666,
  'roasted': 0.49315068906994153,
  'meaty': 0.5789473720534685,
  'pi

# basic data (substructure prediction)

In [2]:
# chembl 推测子结构
getdata = Haha.BasicData(which_data=None)
smiles_list = getdata.readSmiles(filelist=['webScrapping/chembl_smi.txt'])
print(len(smiles_list))

100000


In [3]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# substructure prediction

## single substructure

In [4]:
sub_dict = {
    '0': 'cCc',
    '1': 'CC(C)(C)C',
    '2': 'CC(C)C', 
    '3': 'C1=CCCC1', 
    '4': 'C12CC(CC1)CC2',
    '5': '[A,a]C([A,a])([A,a])[A,a]',
    '6': '[r5]',
    '7': '[r4]',
    '8': 'C(=O)O',
    '9': 'C(=O)N',
    '10': 'CNC',
    '11': 'COOC',
}

od_name_basic = []
for i in sub_dict:
    od_name_basic.append(i)

## making inputs and part of outputs(single substructures)

In [5]:
test = Haha.Transformer2Sub()
test.molData_atomicF(smiles_list, dist_value='exp', atomH=False, failed_mol=None, rseed=0)
test.odData2(smiles_list, sub_dict, od_name=od_name_basic)

transformer2sub
Transformer2OD_tada
transformer2od
1676 mol cannot compute distance, they are: [6, 79, 134, 141, 205, 220, 353, 382, 447, 520, 636, 741, 746, 767, 787, 835, 839, 850, 918, 958, 991, 1039, 1064, 1101, 1127, 1168, 1193, 1237, 1248, 1252, 1277, 1496, 1500, 1517, 1526, 1574, 1584, 1610, 1630, 1659, 1675, 1815, 1831, 1839, 1872, 2031, 2146, 2198, 2233, 2268, 2521, 2592, 2703, 2728, 2756, 2826, 2887, 2940, 2993, 3045, 3128, 3141, 3170, 3178, 3202, 3291, 3310, 3457, 3489, 3529, 3652, 3720, 3779, 3800, 3809, 3818, 3923, 3998, 4033, 4119, 4149, 4153, 4183, 4227, 4327, 4428, 4429, 4475, 4515, 4603, 4697, 4745, 4748, 4762, 4842, 4846, 4852, 4907, 4931, 4976, 5067, 5314, 5346, 5398, 5428, 5434, 5461, 5582, 5636, 5649, 5671, 5738, 5806, 5894, 5961, 6111, 6169, 6176, 6249, 6296, 6361, 6379, 6416, 6439, 6529, 6675, 6686, 6687, 6696, 6927, 6970, 7033, 7109, 7126, 7198, 7226, 7262, 7295, 7318, 7351, 7404, 7439, 7478, 7556, 7561, 7591, 7631, 7672, 7686, 7714, 7890, 7949, 7955, 7978, 8177

## combination of substructures

In [8]:
sub_dict['12'] = '6 and 10'
sub_dict['13'] = '6 and 8'
sub_dict['14'] = '2 and 8'
sub_dict['15'] = '1 or 7'
sub_dict['16'] = '0 or 1'
sub_dict['17'] = '11 or 3'
sub_dict['18'] = '6 and 8 and 9'
sub_dict['19'] = '2 and 5 and 10'
sub_dict['20'] = '0 or 1 or 7'
sub_dict['21'] = '3 or 4 or 11'
sub_dict['22'] = '(6 and 9) or 8'
sub_dict['23'] = '(2 and 8) or 0'


def addSub(od_mat, target_atom=None):
    # target_atom.shape = (samples, sub_basic, max_mol)
    samples = od_mat.shape[0]

    add_od_mat = []
    if target_atom is not None:
        add_target_atom = []
        max_mol = target_atom.shape[-1]

    def sub1_and_sub2(sub1, sub2):
        temp = od_mat[:, sub1] + od_mat[:, sub2] # 0,1,2
        temp -= 1 # -1,0,1
        temp = tf.clip_by_value(temp, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = tf.expand_dims(temp, axis=1) # shape=(samples, 1)

        temp_tar = None
        if target_atom is not None:
            temp_tar = target_atom[:, sub1, :] + target_atom[:, sub2, :] # (samples, max_mol)
            temp_tar *= temp
            temp_tar = tf.expand_dims(temp_tar, axis=1) # shape=(samples, 1, max_mol)    

        return (temp, temp_tar)    

    def sub1_or_sub2(sub1, sub2):
        temp = od_mat[:, sub1] + od_mat[:, sub2] # 0,1,2
        temp = tf.clip_by_value(temp, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = tf.expand_dims(temp, axis=1) # shape=(samples, 1)

        temp_tar = None
        if target_atom is not None:
            temp_tar = target_atom[:, sub1, :] + target_atom[:, sub2, :] # (samples, max_mol)
            temp_tar = tf.expand_dims(temp_tar, axis=1) # shape=(samples, 1, max_mol)    

        return (temp, temp_tar)  

    def sub1_and_sub2_and_sub3(sub1, sub2, sub3):
        temp = od_mat[:, sub1] + od_mat[:, sub2] + od_mat[:, sub3]# 0,1,2,3
        temp -= 2 # -2,-1,0,1
        temp = tf.clip_by_value(temp, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = tf.expand_dims(temp, axis=1) # shape=(samples, 1)

        temp_tar = None
        if target_atom is not None:
            temp_tar = target_atom[:, sub1, :] + target_atom[:, sub2, :] + target_atom[:, sub3, :] # (samples, max_mol)
            temp_tar *= temp
            temp_tar = tf.expand_dims(temp_tar, axis=1) # shape=(samples, 1, max_mol)    

        return (temp, temp_tar)    

    def sub1_or_sub2_or_sub3(sub1, sub2, sub3):
        temp = od_mat[:, sub1] + od_mat[:, sub2] + od_mat[:, sub3] # 0,1,2,3
        temp = tf.clip_by_value(temp, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = tf.expand_dims(temp, axis=1) # shape=(samples, 1)

        temp_tar = None
        if target_atom is not None:
            temp_tar = target_atom[:, sub1, :] + target_atom[:, sub2, :] + target_atom[:, sub3, :]# (samples, max_mol)
            temp_tar = tf.expand_dims(temp_tar, axis=1) # shape=(samples, 1, max_mol)    

        return (temp, temp_tar)   

    def sub1_and_sub2_or_sub3(sub1, sub2, sub3):
        # (sub1 and sub2) or sub3
        temp_and = od_mat[:, sub1] + od_mat[:, sub2] # 0,1,2
        temp_and -= 1
        temp_and = tf.clip_by_value(temp_and, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = temp_and + od_mat[:, sub3]
        temp = tf.clip_by_value(temp, clip_value_min=0, clip_value_max=1) # 0,1 shape=(samples, )
        temp = tf.expand_dims(temp, axis=1) # shape=(samples, 1)

        temp_tar = None
        if target_atom is not None:
            temp_and = tf.expand_dims(temp_and, axis=1) # (samples, 1)
            temp_tar = temp_and*(target_atom[:, sub1, :] + target_atom[:, sub2, :]) + od_mat[:, sub3:sub3+1]*target_atom[:, sub3, :]# (samples, max_mol)
            temp_tar = tf.expand_dims(temp_tar, axis=1) # shape=(samples, 1, max_mol)  

        return(temp, temp_tar)


    ###### sub1 and sub2 #######
    # 6 and 10
    temp, temp_tar = sub1_and_sub2(6, 10)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 6 and 8
    temp, temp_tar = sub1_and_sub2(6, 8)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 2 and 8
    temp, temp_tar = sub1_and_sub2(2, 8)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)

    ###### sub1 or sub2 #######
    # 1 or 7
    temp, temp_tar = sub1_or_sub2(1, 7)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 0 or 1
    temp, temp_tar = sub1_or_sub2(0, 1)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 11 or 3
    temp, temp_tar = sub1_or_sub2(11, 3)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)

    ###### sub1 and sub2 and sub3 ######
    # 6 and 8 and 9
    temp, temp_tar = sub1_and_sub2_and_sub3(6, 8, 9)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 2 and 5 and 10
    temp, temp_tar = sub1_and_sub2_and_sub3(2, 5, 10)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)

    ###### sub1 or sub2 or sub3 ######
    # 0 or 1 or 7
    temp, temp_tar = sub1_or_sub2_or_sub3(0, 1, 7)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # 3 or 4 or 11
    temp, temp_tar = sub1_or_sub2_or_sub3(3, 4, 11)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)

    ###### (sub1 and sub2) or sub3 ######
    # (6 and 9) or 8
    temp, temp_tar = sub1_and_sub2_or_sub3(6, 9, 8)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)
    # (2 and 8) or 0
    temp, temp_tar = sub1_and_sub2_or_sub3(2, 8, 0)
    add_od_mat.append(temp)
    if target_atom is not None:
        add_target_atom.append(temp_tar)

    ###########################################################
    # 把add_od_mat, add_target_atom合到一起
    add_od_mat = tf.concat(add_od_mat, axis=1) 
    if target_atom is not None:
        add_target_atom = tf.concat(add_target_atom, axis=1)
        add_target_atom = tf.clip_by_value(add_target_atom, clip_value_min=0, clip_value_max=1)

        return (add_od_mat, add_target_atom)
    
    else:
        return(add_od_mat)

## make full outputs

In [16]:
test.feature_od = len(sub_dict)

test.od_name = []
for i in sub_dict:
    test.od_name.append(i)

add_od_mat = addSub(test.od_train, target_atom=None)
test.od_train = tf.concat([test.od_train, add_od_mat], axis=1)
print(test.od_train.shape)

add_od_mat, add_target_atom = addSub(test.od_test, target_atom=test.target_atoms)
test.od_test = tf.concat([test.od_test, add_od_mat], axis=1)
print(test.od_train.shape)
test.target_atoms = tf.concat([test.target_atoms, add_target_atom], axis=1)
print(test.target_atoms.shape)


positive = tf.reduce_sum(test.od_train, axis=0)
positive_dict = {}
for each in range(test.feature_od):
    positive_dict[test.od_name[each]] = positive[each].numpy()
print('positive sample distribution in train set:')
print(positive_dict)

positive = tf.reduce_sum(test.od_test, axis=0)
positive_dict1 = {}
for each in range(test.feature_od):
    positive_dict1[test.od_name[each]] = positive[each].numpy()
print('positive sample distribution in test set:')
print(positive_dict1)

total_pos_dict = {}
for each in positive_dict:
    total_pos_dict[each] = positive_dict[each] + positive_dict1[each]
print('positive sample distribution in all data:')
print(total_pos_dict)


(81935, 24)
(16389, 24, 60)
positive sample distribution in train set:
{'0': 5646.0, '1': 4531.0, '2': 21496.0, '3': 408.0, '4': 184.0, '5': 19257.0, '6': 49538.0, '7': 1897.0, '8': 18533.0, '9': 41583.0, '10': 48701.0, '11': 116.0, '12': 29677.0, '13': 10678.0, '14': 6920.0, '15': 6124.0, '16': 10048.0, '17': 524.0, '18': 5296.0, '19': 8475.0, '20': 11549.0, '21': 705.0, '22': 39219.0, '23': 12217.0}
positive sample distribution in test set:
{'0': 1110.0, '1': 913.0, '2': 4325.0, '3': 73.0, '4': 39.0, '5': 3850.0, '6': 9836.0, '7': 354.0, '8': 3757.0, '9': 8238.0, '10': 9717.0, '11': 27.0, '12': 5850.0, '13': 2144.0, '14': 1407.0, '15': 1209.0, '16': 2002.0, '17': 100.0, '18': 1039.0, '19': 1721.0, '20': 2280.0, '21': 139.0, '22': 7868.0, '23': 2453.0}
positive sample distribution in all data:
{'0': 6756.0, '1': 5444.0, '2': 25821.0, '3': 481.0, '4': 223.0, '5': 23107.0, '6': 59374.0, '7': 2251.0, '8': 22290.0, '9': 49821.0, '10': 58418.0, '11': 143.0, '12': 35527.0, '13': 12822.0, '1

## train

In [17]:
test.use_adj_dist = 'both'

hpara = {
            'num_heads': 12,
            'single_attn_dim': 15,
            'num_encoderLayer': 6,
            'num_decoderLayer': 1, 
            'learning_rate': 0.00007, 
            'temperature': 0.7
        }

record_dir='/tf/haha/code/logs/1114/sub-12_15_61'

weight_path = '/tf/haha/save_weight/1114/sub-12_15_61/'

In [20]:
test.modelTrain2(hpara, epochs=200, batch_size=32, record_dir=record_dir, save_threshold=0.9, save_path=weight_path, normal_init=False)

/tf/haha/code/logs/1114/sub-12_15_61
modelBuild2-temperature 0.700000
(81935, 1, 1)
(81935, 60, 71)
(81935, 60, 60)
(81935, 60, 60)
(81935, 1, 1, 60)
(81935, 24)
0 WARNING:tensorflow:Model was constructed with shape (32, 1, 1) for input KerasTensor(type_spec=TensorSpec(shape=(32, 1, 1), dtype=tf.float32, name='input_7'), name='input_7', description="created by layer 'input_7'"), but it was called on an input with incompatible shape (15, 1, 1).
1 2 3 4 5 6 7 8 9 10 weight保存开始：Mon Nov 14 10:11:23 2022 weight保存完成 Mon Nov 14 10:11:23 2022
11 weight保存开始：Mon Nov 14 10:15:03 2022 weight保存完成 Mon Nov 14 10:15:03 2022
12 weight保存开始：Mon Nov 14 10:18:43 2022 weight保存完成 Mon Nov 14 10:18:44 2022
13 14 weight保存开始：Mon Nov 14 10:26:03 2022 weight保存完成 Mon Nov 14 10:26:04 2022
15 weight保存开始：Mon Nov 14 10:29:44 2022 weight保存完成 Mon Nov 14 10:29:44 2022
16 weight保存开始：Mon Nov 14 10:33:25 2022 weight保存完成 Mon Nov 14 10:33:25 2022
17 weight保存开始：Mon Nov 14 10:37:05 2022 weight保存完成 Mon Nov 14 10:37:06 2022
18 wei

(<keras.engine.functional.Functional at 0x7f239f14d190>,
 {'avg': 0.9852966397254473,
  'avg_pcs': 0.9864572410782179,
  'avg_rc': 0.9842843388517698,
  '0': 0.9986480532489226,
  '1': 1.0,
  '2': 0.9998844132350244,
  '3': 0.9241379519620208,
  '4': 0.9473684311036918,
  '5': 1.0,
  '6': 0.9959382826187027,
  '7': 0.9700427955833555,
  '8': 1.0,
  '9': 1.0,
  '10': 1.0,
  '11': 0.981132086930049,
  '12': 0.9971792625051548,
  '13': 0.9941819804831352,
  '14': 1.0,
  '15': 0.9946125360040785,
  '16': 0.9992505880468877,
  '17': 0.9162561587231598,
  '18': 0.997115375584063,
  '19': 0.9997095516372204,
  '20': 0.9969244205309656,
  '21': 0.9366197369222696,
  '22': 0.9987293432926738,
  '23': 0.9993883849993581})

# draw attention

In [ ]:
# make output
out_train = []
out_test = []
for i in range(test.feature_od):
    out_train.append(test.od_train[:, i])
    out_test.append(test.od_test[:, i])
out_train = tuple(out_train)
out_test = tuple(out_test)

In [ ]:
# load model
test.use_adj_dist = 'both'
model = test.modelBuild2(batch_size=32, num_heads=8, single_attn_dim=30, feedforward_dim=240, num_encoderLayer=7, 
                    num_decoderLayer=2, dropout_rate=0.1, lr=0.00002, compile=True, normal_init=False, temperature=0.7)

model.load_weights('/tf/haha/save_weight/1114/od98-8_30_72/')

if len(test.test_input) == test.input_items-1:
    test.test_input = list(test.test_input)
    test.test_input.append(test.od_test)
    test.test_input = tuple(test.test_input)

In [ ]:
preds = model(test.test_input)

# F1 
metrics_dict = {
            'precision': [],
            'recall': []
        }

for each in range(test.feature_od):
    metrics_dict['precision'].append(
        tf.keras.metrics.Precision(name=test.od_name[each]+'_precision'))
    metrics_dict['recall'].append(
        tf.keras.metrics.Recall(name=test.od_name[each]+'_recall'))

evaluate_dict = {}
for i in range(test.feature_od):
    metrics_dict['precision'][i].update_state(
        out_test[i], preds[i])
    metrics_dict['recall'][i].update_state(
        out_test[i], preds[i])

    od = test.od_name[i]
    pcs = metrics_dict['precision'][i].result()
    evaluate_dict[od+'_'+od+'_'+'precision'] = float(pcs)
    rc = metrics_dict['recall'][i].result()
    evaluate_dict[od+'_'+od+'_'+'recall'] = float(rc)

fscore_dict = test.calFscore(evaluate_dict)
print(fscore_dict)

In [ ]:
# attnDraw (Figure9)
selected_od = ['fruity', 'sweet', 'green'] # ...

attn = preds[-2]
attn = [attn[-1]] # if draw each dc layer, delete this line
savepath = '/tf/haha/img_result/1114/od98-8_30_72/'
test.drawAttn(attn, smiles_list, preds, savepath, max_draw=50, addH=False,
                per_head=False, trainORtest='test', attn_c=0.5, posneg='positive', selected_od=selected_od)